# BayesFrag - Tutorial 3: Computation of GMM estimates using OpenQuake

In [1]:
import numpy as np
import pandas as pd
import os
import json
import openquake.hazardlib as oq

## Specify settings

In [ ]:
path_data = os.path.join('data', 'twodim', '')

args = {
    'im_string': 'SAT0_300',
    'GMM': 'BindiEtAl2011',
        }

## Specify earthquake rupture 

The rupture metadata is from ESM: https://esm-db.eu/#/event/IT-2009-0009 

The rupture geometry is from INGV: http://shakemap.ingv.it/shake4/downloadPage.html?eventid=1895389 
The latter is identical to the ESM rupture geometry!

In [5]:
Point = oq.geo.point.Point
PlanarSurface = oq.geo.surface.planar.PlanarSurface
MultiSurface = oq.geo.surface.multi.MultiSurface
BaseRupture = oq.source.rupture.BaseRupture

f = open(path_data + 'rupture.json')
rup_temp = json.load(f)
f.close()
rup_geom_json = rup_temp['features'][0]['geometry']
rup_geom = np.array(rup_geom_json['coordinates'][0][0])[:-1,:]

rupture_surface = PlanarSurface.from_corner_points(
    top_left = Point(rup_geom[0, 0], rup_geom[0, 1], rup_geom[0, 2]),
    top_right = Point(rup_geom[1, 0], rup_geom[1, 1], rup_geom[1, 2]),
    bottom_right = Point(rup_geom[2, 0], rup_geom[2, 1], rup_geom[2, 2]),
    bottom_left = Point(rup_geom[3, 0], rup_geom[3, 1], rup_geom[3, 2]),
)
rupture = BaseRupture(mag = 6.1, rake = -90.0, 
                    tectonic_region_type = 'Active Shallow Crust', 
                    hypocenter = Point(longitude = 13.380, 
                                        latitude = 42.342,
                                        depth = 8.3),
                    surface = rupture_surface)

## Import site information

**Station data**

Below, we import the station data file and print the available attributes, which are:
- id: Station identifier
- Longitude, Latitude in decimal degrees
- vs30: time-averaged shear wave velocity in m/s
- vs30measured: A boolean flag, whether vs30 was measured or deduced from other informations

Besides this information, the station data also contains observed intensity measures (IMs) as processed from the ground motion recordings. In this example, we have data for four IMs: PGA, SA(0.2s), SA(0.3s), and SA(0.6s). Each of these IMs were processed according to two IM definitions: the geometric mean and the rotD50. GMMs are derived for a specific IM definition and if we aim to include the station observations we should extract the correct definition of the employed GMM. This is discussed below.

In [44]:
dfstations = pd.read_csv(path_data + 'stations.csv')
print(dfstations.columns.values)

['id' 'Longitude' 'Latitude' 'vs30' 'vs30measured' 'rotD50_logPGA'
 'geoM_logPGA' 'rotD50_logSAT0_200' 'geoM_logSAT0_200'
 'rotD50_logSAT0_300' 'geoM_logSAT0_300' 'rotD50_logSAT0_600'
 'geoM_logSAT0_600']


**Damage survey data**

Below, we import the station data file and print the available attributes, which are:
- id: Station identifier
- Longitude, Latitude in decimal degrees
- vs30: time-averaged shear wave velocity in the upper-most 30 meters of soil in m/s
- BuildingClass: Used for fragility function estimation (-> see Tutorials 1 and 2)
- DamageState: Used for fragility function estimation (-> see Tutorials 1 and 2)

In [56]:
dfsurvey = pd.read_csv(path_data + 'survey2.csv')
print(dfsurvey.columns.values)

['id' 'Longitude' 'Latitude' 'vs30' 'BuildingClass' 'DamageState']


## Compute GMM estimates

In [22]:
if args['GMM'] == 'BindiEtAl2011':
    gmm = oq.gsim.bindi_2011.BindiEtAl2011()
elif args['GMM'] == 'ChiouYoungs2014Italy':
    gmm = oq.gsim.chiou_youngs_2014.ChiouYoungs2014Italy()

# Extract whether IM is defined for the geometric mean or RotD50
im_definition = gmm.DEFINED_FOR_INTENSITY_MEASURE_COMPONENT.value
if im_definition == 'Average Horizontal':
    obs_str = 'geoM_log' + args['im_string']
elif im_definition == 'Average Horizontal (RotD50)':
    obs_str = 'rotD50_log' + args['im_string']

if args['im_string'] == 'PGA':
    im_list = [oq.imt.PGA()]
else:
    T = float('.'.join( args['im_string'][3:].split('_') )) 
    im_list = [oq.imt.SA(T)]

In [84]:
# Helper functions for bookkeeping

def get_RuptureContext(rupture, sites_mesh, sites_vs30, 
                sites_vs30measured=None, sites_z1pt0=None):

    rctx = oq.contexts.RuptureContext()
    rctx.rjb = rupture.surface.get_joyner_boore_distance(sites_mesh)
    rctx.rrup = rupture.surface.get_min_distance(sites_mesh)
    rctx.vs30 = sites_vs30
    rctx.mag = rupture.mag * np.ones_like(rctx.rjb)
    rctx.rake = rupture.rake * np.ones_like(rctx.rjb)
    rctx.rx = rupture.surface.get_rx_distance(sites_mesh)
    rctx.ztor = rupture.surface.get_top_edge_depth() * np.ones_like(rctx.rjb)
    rctx.dip = rupture.surface.get_dip() * np.ones_like(rctx.rjb)
    if sites_z1pt0 is None:
        rctx.z1pt0 = -7.15/4 * np.log( (sites_vs30**4 + 571**4) / (1360**4 + 571**4) )
    else: 
        rctx.z1pt0 = sites_z1pt0
    if sites_vs30measured is None:
        rctx.vs30measured = False
    else:
        rctx.vs30measured = sites_vs30measured
    return rctx    

def compute_GMM(gmm, im_list, rupture_context):
    n = len(rupture_context.vs30)
    nim = len(im_list)
    mean = np.zeros([nim, n])
    sigma = np.zeros([nim, n])
    tau = np.zeros([nim, n])
    phi = np.zeros([nim, n])
    gmm.compute(rupture_context, im_list, mean, sigma, tau, phi)
    return {'mu_logIM': mean.squeeze(), 
            'tau_logIM': tau.squeeze(), 
            'phi_logIM': phi.squeeze()}

### At the sites of seismic network stations

In [65]:
df = dfstations.copy()
dfgmm = df[['id']].copy()

sites_mesh = oq.geo.mesh.Mesh(df['Longitude'].values, df['Latitude'].values, depths=None)

if args['GMM'] == 'ChiouYoungs2014Italy':
    # Epicentral azimuth is required for spatial correlation model of BodenmannEtAl2023.
    dfgmm['epiazimuth'] = rupture.surface.get_azimuth(sites_mesh).squeeze()

# Extract Observed IM
dfgmm['obs_logIM'] = df[obs_str].values

# Compute GMM estimates
rupture_context = get_RuptureContext(rupture, sites_mesh, 
                                     sites_vs30 = df['vs30'].values, 
                                     sites_vs30measured = df['vs30measured'].values)

res_GMM = compute_GMM(gmm, im_list, rupture_context)

for key in res_GMM.keys():
    dfgmm[key] = res_GMM[key].squeeze()

dfgmm.to_csv(path_data + 'stations_im_' + args['im_string'] + '_gmm_' + args['GMM'] + '.csv', index=False)

In [70]:
dfgmm

,Longitude,Latitude,obs_logIM,mu_logIM,tau_logIM,phi_logIM
id,,,,,,
0,16.271763,40.252750,-5.798707,-6.290024,0.501964,0.66775
1,15.723605,40.601345,-5.672250,-5.724108,0.501964,0.66775
2,13.078598,42.418110,-2.927012,-2.185333,0.501964,0.66775
3,13.339298,42.375530,-0.297195,-0.565073,0.501964,0.66775
4,13.337026,42.373474,-0.114087,-0.565073,0.501964,0.66775
...,...,...,...,...,...,...
59,13.335200,43.062700,-2.841704,-3.347616,0.501964,0.66775
60,13.233100,43.705200,-2.870504,-4.392700,0.501964,0.66775
61,13.401930,42.353880,-0.909393,-0.565073,0.501964,0.66775


### At the sites of surveyed buildings

In [74]:
df = dfsurvey.copy()
dfgmm = df[['id']].copy()

sites_mesh = oq.geo.mesh.Mesh(df['Longitude'].values, df['Latitude'].values, depths=None)

if args['GMM'] == 'ChiouYoungs2014Italy':
    # Epicentral azimuth is required for spatial correlation model of BodenmannEtAl2023.
    dfgmm['epiazimuth'] = rupture.surface.get_azimuth(sites_mesh).squeeze()

# Compute GMM estimates
rupture_context = get_RuptureContext(rupture, sites_mesh, 
                                     sites_vs30 = df['vs30'].values)

res_GMM = compute_GMM(gmm, im_list, rupture_context)

for key in res_GMM.keys():
    dfgmm[key] = res_GMM[key].squeeze()

dfgmm.to_csv(path_data + 'survey2_im_' + args['im_string'] + '_gmm_' + args['GMM'] + '.csv', index=False)

In [77]:
dfgmm

,Longitude,Latitude,BuildingClass,DamageState,mu_logIM,tau_logIM,phi_logIM
id,,,,,,,
0,13.430660,42.302485,A,2,-0.565073,0.501964,0.66775
1,13.288765,42.396429,A,3,-0.677659,0.501964,0.66775
2,13.365190,42.365922,A,3,-0.565073,0.501964,0.66775
3,13.402773,42.348720,A,2,-0.565073,0.501964,0.66775
4,13.400559,42.352410,A,3,-0.565073,0.501964,0.66775
...,...,...,...,...,...,...,...
4195,13.114247,42.017778,C,0,-2.456111,0.501964,0.66775
4196,13.806123,42.479776,C,0,-2.181034,0.501964,0.66775
4197,13.301170,42.288853,C,0,-0.821560,0.501964,0.66775


In [83]:
df = pd.read_csv(path_data + 'gridmap.csv')

dfgmm = df[['id']].copy()

sites_mesh = oq.geo.mesh.Mesh(df['Longitude'].values, df['Latitude'].values, depths=None)

if args['GMM'] == 'ChiouYoungs2014Italy':
    # Epicentral azimuth is required for spatial correlation model of BodenmannEtAl2023.
    dfgmm['epiazimuth'] = rupture.surface.get_azimuth(sites_mesh).squeeze()

# Compute GMM estimates
rupture_context = get_RuptureContext(rupture, sites_mesh, 
                                     sites_vs30 = df['vs30'].values)

res_GMM = compute_GMM(gmm, im_list, rupture_context)

for key in res_GMM.keys():
    dfgmm[key] = res_GMM[key].squeeze()

dfgmm.to_csv(path_data + 'gridmap_im_' + args['im_string'] + '_gmm_' + args['GMM'] + '.csv', index=False)